In [1]:
import math
import json
import random
import os
from tqdm import tqdm

#### GameMap
The gamemap needs to be modified using better data structures to be able to handle longer ops

In [2]:
class Monster:
    def __init__(self, id, x, y, hp, gold, exp):
        self.id = id
        self.x, self.y, self.hp, self.gold, self.exp = x, y, hp, gold, exp
        
class Gamemap:
    def __init__(self, width, height, monsters):
        self.width, self.height, self.monsters = width, height, monsters
        
    def getMonsters(self, x, y, range):
        monsters = []
        for monster in self.monsters:
            if (monster.x - x)**2 + (monster.y - y)**2 < range**2:
                monsters.append(monster)
        return monsters
    
    def attackMonster(self, id, power):
        targetId, eraseId = -1, -1
        gold, exp = 0, 0
        for i in range(len(self.monsters)):
            if self.monsters[i].id == id:
                self.monsters[i].hp -= power
                if self.monsters[i].hp <= 0:
                    gold, exp = self.monsters[i].gold, self.monsters[i].exp
                    eraseId = i
                    break
        if eraseId != -1:
            del self.monsters[eraseId]
        return id, gold, exp
        

### Hero base class
Extend the same and implement `moveStrategy()`, `attackStrategy()` and `chooseMove()`

In [3]:
class Hero:
    def __init__(self, init_x, init_y, init_speed, init_power, init_range, speed_alpha, power_alpha, range_alpha, gamemap):
        self.base_speed, self.base_power, self.base_range = init_speed, init_power, init_range
        self.speed_alpha, self.power_alpha, self.range_alpha = speed_alpha, power_alpha, range_alpha
        self.x, self.y, self.speed, self.power, self.range = init_x, init_y, init_speed, init_power, init_range
        self.gold, self.exp, self.level = 0, 0, 0
        self.gamemap = gamemap
        self.logs = []
        
    def moveStrategy(self):
        pass
    
    def attackStrategy(self):
        pass
    
    def chooseMove(self): # True if movement, False for attack
        pass
    
    def recomputeStats(self):
        if self.exp >= 1000 + self.level * (self.level + 1):
            self.exp -= 1000 + self.level * (self.level + 1)
            self.level += 1
            self.speed = math.floor(self.base_speed * (1 + self.level * self.speed_alpha / 100))
            self.power = math.floor(self.base_power * (1 + self.level * self.power_alpha / 100))
            self.range = math.floor(self.base_range * (1 + self.level * self.range_alpha / 100))
        
    def move(self):
        nx, ny = self.moveStrategy()
        self.logs.append({
            "type" : "move", 
            "target_x" : nx,
            "target_y" : ny
        })
        self.x, self.y = nx, ny
        
    def attack(self):
        target, gold, exp = self.attackStrategy()
        if target == -1:
            return False
        self.logs.append({
            "type" : "attack",
            "target_id" : target
        })
        self.gold += gold
        self.exp += exp
        self.recomputeStats()
        return True
        
    def simulate(self, num_turns, output_file):
        self.logs = []
        i = 0
        while i < num_turns:
            nextMove = self.chooseMove()
            if nextMove : # True if movement, False for attack
                self.move()
                i += 1
            else :
                if self.attack():
                    i += 1
                
                
        with open(output_file, 'w') as f:
            json.dump({
                "moves" : self.logs
            }, f, indent = 4)

In [125]:
class RandomHero(Hero):
    def moveStrategy(self):
        d = random.choice(range(1, self.speed + 1))
        dx, dy = random.choice([(0, d), (d, 0), (0, -d), (-d, 0)])
        nx, ny = self.x + dx, self.y + dy
        while(nx > self.gamemap.width or nx < 0 or ny > self.gamemap.height or ny < 0):
            d = random.choice(range(1, self.speed + 1))
            dx, dy = random.choice([(0, d), (d, 0), (0, -d), (-d, 0)])
            nx, ny = self.x + dx, self.y + dy
        return nx, ny
    
    def attackStrategy(self):
        monsters = self.gamemap.getMonsters(self.x, self.y, self.range)
        if len(monsters) == 0:
            return (-1, 0, 0)
        monster = random.choice(monsters)
        return self.gamemap.attackMonster(monster.id, self.power)
    
    def chooseMove(self):
        P_ATTACK = 0.8
        return random.choices([True, False], weights = [1 - P_ATTACK, P_ATTACK], k = 1)[0]

In [126]:
def processFile(filename):
    data = {}
    with open(f"./ip/{filename}", "r") as f:
        data = json.load(f)
    monsters = []
    for i, md in enumerate(data["monsters"]):
        monster = Monster(i, md['x'], md['y'], md['hp'], md['gold'], md['exp'])
        monsters.append(monster)
    gamemap = Gamemap(data["width"], data["height"], monsters)
    
    hero = RandomHero(
        data["start_x"], data["start_y"], 
        data["hero"]["base_speed"], data["hero"]["base_power"], data["hero"]["base_range"],
        data["hero"]["level_speed_coeff"], data["hero"]["level_power_coeff"], data["hero"]["level_range_coeff"], gamemap
    )
    hero.simulate(data["num_turns"], f"./op/{filename}")

In [127]:
files = [f for f in os.listdir("./ip")]

In [128]:
for f in tqdm(files) :
    processFile(f)

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:52<00:00,  2.08s/it]
